In [21]:
import pandas as pd
import yfinance as yf
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
from matplotlib import rcParams
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import minimize
import seaborn as sns
import matplotlib.dates as mdates
from portfolio_utils import *
# import ace_tools as tools; 

# 配置中文字体
plt.rcParams['font.family'] = 'Heiti TC'

# plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 150
plt.rcParams.update({'font.size': 12})

In [22]:

# 从 Excel 文件中读取数据
file_path = 'Stocks.xlsx'  # 替换为你的Excel文件路径
etf_data = pd.read_excel(file_path, sheet_name='ETF')

# 从Excel中获取股票代码和对应的美股指数代码
etf_symbols = etf_data['Stock Symbol'].tolist()
etf_names = etf_data['Stock Name'].tolist()
index_symbols = etf_data['Target'].tolist()

start_date = '2020-08-23'
end_date = '2024-08-24'

window = 10
initial_capital = 37000

# 下载A股ETF数据
etf_prices = get_stock_price(etf_symbols, start_date, end_date)
# 下载美股指数数据
index_prices = get_stock_price(index_symbols, start_date, end_date)

# 重命名列，方便处理
etf_prices.columns = etf_data['Stock Name']
index_prices.columns = etf_data['Target']

# 将美股指数向前移动一天，以匹配中国交易时间（即将美股前一天的数据与中国当天对齐）
index_prices_shifted = index_prices.shift(1)

# 去除ETF成立时间之前的NaN数据
etf_prices.dropna(how='all', inplace=True)

# 去除美股指数缺失值
index_prices_shifted.dropna(how='all', inplace=True)

# 将日期对齐，只保留中国交易日
common_dates = etf_prices.index.intersection(index_prices_shifted.index)
etf_prices = etf_prices.loc[common_dates]
index_prices_shifted = index_prices_shifted.loc[common_dates]
# 用前一个有效值填充 NaN
etf_prices.ffill(inplace=True)
index_prices_shifted.ffill(inplace=True)

index_prices = index_prices_shifted
dates = etf_prices.index[window-1:]
SSINDX = yf.download('000001.SS', start_date, end_date)['Close']
SSINDX = SSINDX[common_dates]
benchmark = SSINDX / SSINDX.iloc[0] * initial_capital


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [23]:
# 初始分配权重
def initial_weights(stock_symbols):
    num_stocks = len(stock_symbols)
    return np.ones(num_stocks) / num_stocks

# 重新分配权重
def rebalance_weights(diff_z_scores, date, initial_weight):
    ranked_stocks = diff_z_scores.loc[date].sort_values()
    num_stocks = len(ranked_stocks)
    new_weights = np.zeros(num_stocks)
    
    # 根据排序结果重新分配权重
    for i, stock in enumerate(ranked_stocks.index):
        new_weights[i] = initial_weight * (1 + ranked_stocks.index.get_loc(stock) / num_stocks)
    new_weights /= new_weights.sum()  # 归一化权重

    return new_weights

def rebalance_rank_portfolio(etf_prices, index_prices, window, initial_capital):
    transaction_cost=0.005
    lot_size=100
    min_trade_volume = 200
    diff_z_scores = calculate_diff_z_score(etf_prices, index_prices, window=5)
    
    num_etfs = len(etf_prices.columns)
    initial_weight = 1 / num_etfs
    weights = initial_weights(etf_prices.columns)
    portfolio_value = initial_capital
    portfolio_values = [initial_capital]
    positions = np.zeros((len(etf_prices.index[window-1:]), num_etfs))  # 存储每个交易日的仓位
    adjustments = np.zeros_like(positions)  # 存储每个交易日的调整信号
    current_positions = np.zeros(num_etfs)
    positions[0] = current_positions
    cash = initial_capital
    buy_count = []
    sell_count = []
    annual_buy_count = 0
    annual_sell_count = 0
    current_year = etf_prices.index[window].year
    
    for t, date in enumerate(etf_prices.index[window:], start=1):
        if date.year != current_year:
            buy_count.append(annual_buy_count)
            sell_count.append(annual_sell_count)
            annual_buy_count = 0
            annual_sell_count = 0
            current_year = date.year
        
        # 根据Z-score排序重新分配权重
        new_weights = rebalance_weights(diff_z_scores, date, initial_weight)
        
        # 根据新的权重进行买卖股票
        for i, etf in enumerate(etf_prices.columns):
            current_price = etf_prices[etf].loc[date]
            desired_position = portfolio_value * new_weights[i] // current_price
            
            # 检查交易量是否达到最低交易量限制
            trade_volume = abs(desired_position - current_positions[i]) * current_price
            if trade_volume < min_trade_volume:
                continue  # 如果交易量小于最低限制，则跳过此交易
            
            if desired_position > current_positions[i]:
                buy_amount = (desired_position - current_positions[i]) // lot_size * lot_size
                if buy_amount > 0 and cash >= buy_amount * current_price * (1 + transaction_cost):
                    current_positions[i] += buy_amount
                    cash -= buy_amount * current_price * (1 + transaction_cost)
                    annual_buy_count += 1
            elif desired_position < current_positions[i]:
                sell_amount = (current_positions[i] - desired_position) // lot_size * lot_size
                if sell_amount > 0:
                    current_positions[i] -= sell_amount
                    cash += sell_amount * current_price * (1 - transaction_cost)
                    annual_sell_count += 1

        weights = new_weights
        portfolio_value = cash + np.sum(current_positions * etf_prices.loc[date])
        portfolio_values.append(portfolio_value)
        positions[t] = current_positions  # 记录仓位
        
    buy_count.append(annual_buy_count)
    sell_count.append(annual_sell_count)

    return portfolio_values, weights, positions, adjustments, buy_count, sell_count
    

In [24]:
portfolio_values, weights, positions, adjustments, buy_count, sell_count = rebalance_rank_portfolio(etf_prices, index_prices, window, initial_capital)
print(f'Final Portfolio Value: {portfolio_values[-1]}')

Final Portfolio Value: 46046.13151293993


In [25]:
summary_total = pnl_summary_total(portfolio_values, dates, positions, etf_prices, window=window)
summary_total

,Total Return %,Annualized Return %,Sharpe Ratio,Max Drawdown %,Turnover %,Fitness
0,24.45,5.68,1.26,5.95,0.91,0.85


In [26]:
summary = pnl_summary(portfolio_values, dates, buy_count, sell_count)
summary

,Start Value,End Value,Return Std,Annual Return %,Sharpe Ratio,Max Drawdown %,Buy Count,Sell Count
Year,,,,,,,,
2020,37000.00,37151.68,0.00,0.41,0.15,1.09,5,1
2021,37168.48,38491.38,0.00,3.56,1.22,1.72,1,9
2022,38515.88,37657.76,0.00,-2.23,-0.50,3.59,11,8
2023,37651.76,41789.12,0.00,10.99,2.61,2.59,3,14
2024,41797.32,46046.13,0.01,10.17,1.25,5.95,11,29


In [27]:
pnl_visualization(dates, portfolio_values, benchmark)

In [28]:
plot_historical_positions(dates, positions, etf_names, etf_prices, window)